In [1]:
import numpy as np
from scipy.io import loadmat
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense
from tensorflow.keras import Model, Input

# Settings

## General settings

In [9]:
settings = {}
settings['cdrtype'] = 0
settings['diffh'] = 9
settings['method'] = 'multiparametric'

def sett2str(sett):
    sett_str = ''
    for key, value in sett.items():
        sett_str += '%s%s_' % (key, value)
    return sett_str[:-1]

## Path settings

In [10]:
save_path = 'results'
load_path = os.path.join('results', 'timeseries')


# Load timeseries

In [11]:
ts = None

for cdrtype in [1, 2, 3, 4]:
    settings['cdrtype'] = cdrtype
    
    file_path = os.path.join(load_path, 'ts_%s.mat' % sett2str(settings))
    
    if settings['method'] == 'multiparametric':
        ts_i = loadmat(file_path)['ts']
    else:
        ts_i = np.expand_dims(loadmat(file_path)['ts'], axis=2)
    
    if ts is None:
        ts = ts_i
    else:
        ts = np.concatenate((ts, ts_i), axis=2)

y = np.ones((ts.shape[0],))
y[int(y.shape[0]/2):] = 0

print(ts.shape, y.shape)

(20000, 42, 8) (20000,)


# Train-Test split

In [12]:
ts_tr, ts_te, y_tr, y_te = train_test_split(ts, y, test_size=0.2, random_state=1)

# LSTM

## Define network

In [13]:
n_sample_per_user = ts.shape[1]
n_ch = ts.shape[2]

input_layer = Input(
    shape=(n_sample_per_user, n_ch),
    name='input'
)

lstm_layer = LSTM(
    units=int(n_sample_per_user/6),
    activation=None,
    use_bias=True,
    name='lstm_1'
)(input_layer)

dense = Dense(
    units=1,
    activation='sigmoid',
    use_bias=True,
    name='dense_1'
)(lstm_layer)

    
mdl = Model(inputs=input_layer, outputs=dense)

mdl.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 42, 8)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 7)                 448       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 8         
Total params: 456
Trainable params: 456
Non-trainable params: 0
_________________________________________________________________


## Compile

In [14]:
mdl.compile(loss='binary_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

## Fit

In [15]:
mdl.fit(ts_tr, y_tr,
        validation_data=(ts_te, y_te),
        epochs=3,
        verbose=1)

Epoch 1/3
500/500 [==============================] - 25s 42ms/step - loss: 0.5412 - accuracy: 0.6738 - val_loss: 0.3548 - val_accuracy: 0.8543
Epoch 2/3
500/500 [==============================] - 18s 36ms/step - loss: 0.3512 - accuracy: 0.8520 - val_loss: 0.3486 - val_accuracy: 0.8528
Epoch 3/3
500/500 [==============================] - 18s 35ms/step - loss: 0.3470 - accuracy: 0.8556 - val_loss: 0.3670 - val_accuracy: 0.8555
